In [24]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# (c) Tariq Rashid, 2016
# license is GPLv2

In [25]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [54]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        #self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.wih = numpy.random.normal(0.0, 0.5, (self.hnodes, self.inodes))
        #self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        self.who = numpy.random.normal(0.0, 0.5, (self.hnodes, self.inodes))
        print("wih: ", self.wih)
        print("who: ", self.who)

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        print("INPUTS")
        print(inputs)
        print("===========================")
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        print("HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)")
        print(hidden_inputs)
        print("===========================")
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        print("HIDDEN OUTPUTS (después de aplicar la función sigmoide)")
        print(hidden_outputs)
        print("===========================")
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        print("FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)")
        print(final_inputs)
        print("===========================")
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        print("FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)")
        print(final_outputs)
        print("===========================")
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        print("ERROR (target - valor de los final outputs)")
        print(output_errors)
        print("===========================")
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        print("Peso actualizado del output layer: ", self.who)
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        print("Peso actualizado del hidden layer: ", self.wih)
        
        print("============= FINAL DE FASE DE ENTRENAMIENTO ==============")
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [55]:
# number of input, hidden and output nodes
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

wih:  [[-0.41103776 -0.23514698 -0.20896141]
 [-0.28968639 -0.20417809  0.81662542]
 [ 0.28036671 -0.07076208  0.36728809]]
who:  [[ 0.05734282 -0.19655358 -0.17788809]
 [ 0.57553265 -0.66679564  0.13008393]
 [ 0.15773582 -0.28793691 -0.26157918]]


In [56]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_dataset/contador.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [57]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 100

for e in range(epochs):
    # go through all records in the training data set
    contador = 1
    for record in training_data_list:
        print("Iteración #", e)
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[0:3]) / 1.0) + 0.00
        # inputs = (numpy.asfarray(all_values[0:3]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        # targets = numpy.zeros(output_nodes) + 0.01
        target_values = training_data_list[contador].split(',')
        targets = (numpy.asfarray(target_values[0:3]) / 1.0) + 0.00
        print("inputs -> ", inputs)
        print("targets -> ", targets)
        # all_values[0] is the target label for this record
        # targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        contador += 1
        if(contador == 8):
            contador = 0
        pass
    pass

Iteración # 0
inputs ->  [ 0.  0.  0.]
targets ->  [ 0.  0.  1.]
INPUTS
[[ 0.]
 [ 0.]
 [ 0.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[ 0.]
 [ 0.]
 [ 0.]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.5]
 [ 0.5]
 [ 0.5]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[-0.15854943]
 [ 0.01941047]
 [-0.19589014]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.46044547]
 [ 0.50485247]
 [ 0.45118347]]
ERROR (target - valor de los final outputs)
[[-0.46044547]
 [-0.50485247]
 [ 0.54881653]]
Peso actualizado del output layer:  [[ 0.05734282 -0.19655358 -0.17788809]
 [ 0.57553265 -0.66679564  0.13008393]
 [ 0.15773582 -0.28793691 -0.26157918]]
Peso actualizado del hidden layer:  [[-0.41103776 -0.23514698 -0.20896141]
 [-0.28968639 -0.20417809  0.81662542]
 [ 0.28036671 -0.07076208  0.36728809]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 0
inputs ->  [ 0.  0.  1.]
targets -

ERROR (target - valor de los final outputs)
[[ 0.53498211]
 [ 0.4936867 ]
 [ 0.54783105]]
Peso actualizado del output layer:  [[ 0.05831128 -0.18828184 -0.16397974]
 [ 0.57637143 -0.66107085  0.13400749]
 [ 0.17038098 -0.29329303 -0.25740364]]
Peso actualizado del hidden layer:  [[-0.39701682 -0.2227476  -0.21012161]
 [-0.31760515 -0.22405158  0.82125614]
 [ 0.26314313 -0.07882347  0.38280045]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 1
inputs ->  [ 1.  1.  1.]
targets ->  [ 0.  0.  0.]
INPUTS
[[ 1.]
 [ 1.]
 [ 1.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[-0.82988604]
 [ 0.27959941]
 [ 0.56712011]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.30366917]
 [ 0.56944801]
 [ 0.63809839]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[-0.18525805]
 [-0.10766967]
 [-0.27046347]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.4538175 ]
 [ 0.47310856]
 [ 0.43279332]]
ERR

 [-0.25938623]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.44720998]
 [ 0.47208744]
 [ 0.43551459]]
ERROR (target - valor de los final outputs)
[[ 0.55279002]
 [-0.47208744]
 [-0.43551459]]
Peso actualizado del output layer:  [[ 0.04540381 -0.20240218 -0.16868666]
 [ 0.58340181 -0.6513      0.14368789]
 [ 0.19403291 -0.29591034 -0.24574531]]
Peso actualizado del hidden layer:  [[-0.39892304 -0.22531426 -0.23167667]
 [-0.31926901 -0.22045226  0.85657046]
 [ 0.25206314 -0.08250207  0.40796369]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 4
inputs ->  [ 1.  0.  0.]
targets ->  [ 1.  0.  1.]
INPUTS
[[ 1.]
 [ 0.]
 [ 0.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[-0.39892304]
 [-0.31926901]
 [ 0.25206314]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.40157112]
 [ 0.4208539 ]
 [ 0.56268424]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[-0.15150657]
 [ 0.03210701]
 [-

HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[-0.63716606]
 [ 0.54148281]
 [ 0.65527586]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.34588744]
 [ 0.63215729]
 [ 0.65819838]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[-0.19422643]
 [-0.13401475]
 [-0.27990573]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.45159546]
 [ 0.46654637]
 [ 0.43047689]]
ERROR (target - valor de los final outputs)
[[ 0.54840454]
 [ 0.53345363]
 [-0.43047689]]
Peso actualizado del output layer:  [[ 0.05680326 -0.18524246 -0.14702496]
 [ 0.57426229 -0.66267465  0.13106953]
 [ 0.20242825 -0.30095834 -0.2425866 ]]
Peso actualizado del hidden layer:  [[-0.39399836 -0.22303804 -0.23183928]
 [-0.33218314 -0.22364859  0.85852617]
 [ 0.24051107 -0.08492858  0.41898161]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 5
inputs ->  [ 1.  1.  0.]
targets ->  [ 1.  1.  1.]
INPUTS
[[ 1.]
 [ 1.]
 [ 0.]]
HID

FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[-0.18874414]
 [-0.13272925]
 [-0.27935684]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.45295355]
 [ 0.46686632]
 [ 0.43061146]]
ERROR (target - valor de los final outputs)
[[ 0.54704645]
 [ 0.53313368]
 [-0.43061146]]
Peso actualizado del output layer:  [[ 0.05740309 -0.1827684  -0.14100897]
 [ 0.57478759 -0.66109503  0.13238513]
 [ 0.20960019 -0.30489617 -0.24098874]]
Peso actualizado del hidden layer:  [[-0.39086923 -0.22073228 -0.23803769]
 [-0.33882949 -0.22676324  0.86837111]
 [ 0.23436364 -0.0872928   0.42793515]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 6
inputs ->  [ 1.  1.  0.]
targets ->  [ 1.  1.  1.]
INPUTS
[[ 1.]
 [ 1.]
 [ 0.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[-0.61160152]
 [-0.56559274]
 [ 0.14707084]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.35169396]
 [ 0.3622544 ]
 [ 0.53670158]]
FIN

...

Iteración # 93
inputs ->  [ 1.  1.  0.]
targets ->  [ 1.  1.  1.]
INPUTS
[[ 1.]
 [ 1.]
 [ 0.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[ 0.59856274]
 [-1.29554388]
 [-0.2128361 ]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.64532742]
 [ 0.21491593]
 [ 0.44699093]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[ 0.05250893]
 [ 0.31794907]
 [ 0.5304719 ]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.51312422]
 [ 0.57882434]
 [ 0.62959317]]
ERROR (target - valor de los final outputs)
[[ 0.48687578]
 [ 0.42117566]
 [ 0.37040683]]
Peso actualizado del output layer:  [[ 0.07145841 -0.21786241  0.11905601]
 [ 0.52595911 -0.53781563  0.21055993]
 [ 1.33936279 -0.93584888 -0.29693399]]
Peso actualizado del hidden layer:  [[ 0.19874513  0.43426045 -1.83622493]
 [-0.77005077 -0.54841407  2.26693356]
 [ 0.02550207 -0.23652568  1.1477241 ]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
It

inputs ->  [ 0.  0.  1.]
targets ->  [ 0.  1.  0.]
INPUTS
[[ 0.]
 [ 0.]
 [ 1.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[-1.87683747]
 [ 2.29883042]
 [ 1.15381265]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.13275255]
 [ 0.90878013]
 [ 0.76020662]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[-0.11672977]
 [-0.27744217]
 [-0.89804328]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.47085065]
 [ 0.43108097]
 [ 0.28945277]]
ERROR (target - valor de los final outputs)
[[-0.47085065]
 [ 0.56891903]
 [-0.28945277]]
Peso actualizado del output layer:  [[ 0.071506   -0.23196645  0.11126465]
 [ 0.52181341 -0.54940647  0.20070261]
 [ 1.36736916 -0.94194589 -0.294056  ]]
Peso actualizado del hidden layer:  [[ 0.19481287  0.43190012 -1.87836394]
 [-0.76012175 -0.5385218   2.29940493]
 [ 0.02407628 -0.23888425  1.15649071]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 95
in

targets ->  [ 1.  0.  1.]
INPUTS
[[ 1.]
 [ 0.]
 [ 0.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[ 0.20461513]
 [-0.76415231]
 [ 0.02384515]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.55097605]
 [ 0.31774543]
 [ 0.50596101]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[ 0.01559396]
 [ 0.22393419]
 [ 0.30947719]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.50389841]
 [ 0.55575077]
 [ 0.57675764]]
ERROR (target - valor de los final outputs)
[[ 0.49610159]
 [-0.55575077]
 [ 0.42324236]]
Peso actualizado del output layer:  [[ 0.06614502 -0.23765841  0.10804095]
 [ 0.52668852 -0.54094271  0.20875819]
 [ 1.3895994  -0.9575682  -0.30021262]]
Peso actualizado del hidden layer:  [[ 0.21273595  0.45148223 -1.91523082]
 [-0.76897697 -0.54930241  2.32238434]
 [ 0.0191088  -0.24105395  1.16466471]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 96
inputs ->  [ 1.  0.  1.]
ta

 [ 1.41353968 -0.96223358 -0.29608854]]
Peso actualizado del hidden layer:  [[ 0.22119635  0.46282852 -1.94341586]
 [-0.7748434  -0.55336319  2.33826965]
 [ 0.02366397 -0.24221146  1.17653216]]
============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 97
inputs ->  [ 1.  1.  0.]
targets ->  [ 1.  1.  1.]
INPUTS
[[ 1.]
 [ 1.]
 [ 0.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[ 0.68402487]
 [-1.32820659]
 [-0.21854748]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.66463641]
 [ 0.20945617]
 [ 0.44557956]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[ 0.05976832]
 [ 0.32863075]
 [ 0.60196469]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.51493763]
 [ 0.58142618]
 [ 0.64610567]]
ERROR (target - valor de los final outputs)
[[ 0.48506237]
 [ 0.41857382]
 [ 0.35389433]]
Peso actualizado del output layer:  [[ 0.07621704 -0.22465186  0.12605251]
 [ 0.51964347 -0.53275251  0.21285779]
 [

============= FINAL DE FASE DE ENTRENAMIENTO ==============
Iteración # 99
inputs ->  [ 0.  0.  0.]
targets ->  [ 0.  0.  1.]
INPUTS
[[ 0.]
 [ 0.]
 [ 0.]]
HIDDEN INPUTS (multiplicación cruz de inputs directos con peso wih)
[[ 0.]
 [ 0.]
 [ 0.]]
HIDDEN OUTPUTS (después de aplicar la función sigmoide)
[[ 0.5]
 [ 0.5]
 [ 0.5]]
FINAL INPUTS (multiplicación cruz de hidden outputs con peso who)
[[-0.01280612]
 [ 0.09680682]
 [ 0.07500044]]
FINAL OUTPUTS (después de aplicar función sigmoide a los final inputs)
[[ 0.49679851]
 [ 0.52418282]
 [ 0.51874132]]
ERROR (target - valor de los final outputs)
[[-0.49679851]
 [-0.52418282]
 [ 0.48125868]]
Peso actualizado del output layer:  [[ 0.08281203 -0.23276486  0.1243406 ]
 [ 0.52218598 -0.53798909  0.20941675]
 [ 1.43428876 -0.9797846  -0.30450329]]
Peso actualizado del hidden layer:  [[ 0.23376617  0.47685347 -1.98412325]
 [-0.77607658 -0.55463637  2.36977997]
 [ 0.02341821 -0.24346696  1.18239443]]
============= FINAL DE FASE DE ENTRENAMIENTO ==

In [30]:
# load the mnist test data CSV file into a list
test_data_file = open("mnist_dataset/contador.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [38]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    # inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    inputs = (numpy.asfarray(all_values[0:3]) / 1.0) + 0.00
    # query the network
    outputs = n.query(inputs)
    print("Outputs -> ", outputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

Outputs ->  [[ 0.43989384]
 [ 0.51993662]
 [ 0.65679279]]
Outputs ->  [[ 0.3811616 ]
 [ 0.47311188]
 [ 0.15361066]]
Outputs ->  [[ 0.30698364]
 [ 0.53523162]
 [ 0.80216717]]
Outputs ->  [[ 0.35260061]
 [ 0.47398271]
 [ 0.16097631]]
Outputs ->  [[ 0.74257478]
 [ 0.54444246]
 [ 0.82155565]]
Outputs ->  [[ 0.5920998 ]
 [ 0.48284944]
 [ 0.20954849]]
Outputs ->  [[ 0.68180777]
 [ 0.55284104]
 [ 0.87408349]]
Outputs ->  [[ 0.47099956]
 [ 0.48239088]
 [ 0.20872126]]


In [32]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.0
